<a href="https://colab.research.google.com/github/MpRonald/Spark/blob/main/Spark_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# criando variaveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
# importando a lib findspark
import findspark
findspark.init()

In [4]:
# iniciando uma sessão spark
from pyspark.sql import SparkSession

spark = SparkSession\
            .builder.master('local[*]')\
            .appName('Iniciando com Spark')\
            .config('spark.ui.port', '4050')\
            .getOrCreate()

In [5]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   


In [6]:
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [7]:
data = [('Zeca','35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']
df = spark.createDataFrame(data, colNames)
df

DataFrame[Nome: string, Idade: string]

In [8]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import zipfile
zipfile.ZipFile('/content/drive/MyDrive/Datasets/movie_rates.zip', 'r').extractall('/content/drive/MyDrive/Datasets')

In [19]:
path = '/content/drive/MyDrive/Datasets/Top_10000_Movies.csv'
movies = spark.read.csv(path, sep=',', header=True, inferSchema=True)

In [20]:
movies.count()

10036

In [21]:
movies.limit(5).toPandas()

,_c0,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
0,0,580489,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000,97,None
1,1,524434,en,Eternals,3365.535,2021-11-03,7.1,622,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000,157,In the beginning...
2,2,438631,en,Dune,2911.423,2021-09-15,8.0,3632,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356,155,"Beyond fear, destiny awaits."
3,3,796499,en,Army of Thieves,2552.437,2021-10-27,6.9,555,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0,127,"Before Vegas, one locksmith became a legend."
4,4,550988,en,Free Guy,1850.47,2021-08-11,7.8,3493,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766,115,Life's too short to be a background character.


In [22]:
movies.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- tagline: string (nullable = true)



In [23]:
movies.show()

+---+------+-----------------+------------------------------------+----------+------------+------------+----------+--------------------+--------------------+--------------------+-----------------+--------------------+
|_c0|    id|original_language|                      original_title|popularity|release_date|vote_average|vote_count|               genre|            overview|             revenue|          runtime|             tagline|
+---+------+-----------------+------------------------------------+----------+------------+------------+----------+--------------------+--------------------+--------------------+-----------------+--------------------+
|  0|580489|               en|                Venom: Let There ...|  5401.308|  2021-09-30|         6.8|      1736|['Science Fiction...|After finding a h...|           424000000|               97|                null|
|  1|524434|               en|                            Eternals|  3365.535|  2021-11-03|         7.1|       622|['Action', 'A

In [24]:
movies.columns

['_c0',
 'id',
 'original_language',
 'original_title',
 'popularity',
 'release_date',
 'vote_average',
 'vote_count',
 'genre',
 'overview',
 'revenue',
 'runtime',
 'tagline']

In [44]:
dataset = movies.select('original_title', 'original_language', 'vote_average', 'runtime')
dataset.show()

+------------------------------------+-----------------+------------+-----------------+
|                      original_title|original_language|vote_average|          runtime|
+------------------------------------+-----------------+------------+-----------------+
|                Venom: Let There ...|               en|         6.8|               97|
|                            Eternals|               en|         7.1|              157|
|                                Dune|               en|         8.0|              155|
|                     Army of Thieves|               en|         6.9|              127|
|                            Free Guy|               en|         7.8|              115|
|                 Gunpowder Milkshake|               en|         6.5|              114|
|                               Venom|               en|         6.8|              112|
|                     Halloween Kills|               en|         7.0|              105|
|                W lesie dziś ni

In [45]:
# mudando o tipo da coluna vote_average
dataset = dataset.withColumn('mean_votes',
                              dataset['vote_average']\
                              .cast('float'))\
                              .drop('vote_average')

# mudando o tipo da coluna runtime
dataset = dataset.withColumn('time',
                              dataset['runtime']\
                              .cast('int'))\
                              .drop('runtime')                              

In [46]:
dataset.show()

+------------------------------------+-----------------+----------+----+
|                      original_title|original_language|mean_votes|time|
+------------------------------------+-----------------+----------+----+
|                Venom: Let There ...|               en|       6.8|  97|
|                            Eternals|               en|       7.1| 157|
|                                Dune|               en|       8.0| 155|
|                     Army of Thieves|               en|       6.9| 127|
|                            Free Guy|               en|       7.8| 115|
|                 Gunpowder Milkshake|               en|       6.5| 114|
|                               Venom|               en|       6.8| 112|
|                     Halloween Kills|               en|       7.0| 105|
|                W lesie dziś nie ...|               pl|       4.1|  96|
|                     American Badger|               en|       6.3|  88|
|                 The Addams Family 2|             

In [47]:
dataset.printSchema()

root
 |-- original_title: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- mean_votes: float (nullable = true)
 |-- time: integer (nullable = true)



In [50]:
# filtro
dataset.filter(dataset.mean_votes > 5.).show()

+------------------------------------+-----------------+----------+----+
|                      original_title|original_language|mean_votes|time|
+------------------------------------+-----------------+----------+----+
|                Venom: Let There ...|               en|       6.8|  97|
|                            Eternals|               en|       7.1| 157|
|                                Dune|               en|       8.0| 155|
|                     Army of Thieves|               en|       6.9| 127|
|                            Free Guy|               en|       7.8| 115|
|                 Gunpowder Milkshake|               en|       6.5| 114|
|                               Venom|               en|       6.8| 112|
|                     Halloween Kills|               en|       7.0| 105|
|                     American Badger|               en|       6.3|  88|
|                 The Addams Family 2|               en|       7.4|  93|
|劇場版 七つの大罪 光に呪われし者たち|               ja|       8.4|n

In [59]:
# verificando o valor maximo para mean_votes e time
from pyspark.sql.functions import max
dataset.agg(max('mean_votes').alias('mean_max_voltes')).show()

+---------------+
|mean_max_voltes|
+---------------+
|            9.5|
+---------------+



In [60]:
dataset.agg(max('time').alias('max_time')).show()

+----------+
|  max_time|
+----------+
|1073394593|
+----------+



In [64]:
# retornando filme com valor maximo de time e mean_votes
dataset.filter(dataset.mean_votes == 9.5).show(),
dataset.filter(dataset.time == 1073394593).show()

+--------------------+-----------------+----------+----+
|      original_title|original_language|mean_votes|time|
+--------------------+-----------------+----------+----+
|       Ebola Zombies|               zh|       9.5|  90|
|Bob Esponja: Días...|               es|       9.5|   0|
+--------------------+-----------------+----------+----+

+--------------+-----------------+----------+----------+
|original_title|original_language|mean_votes|      time|
+--------------+-----------------+----------+----------+
|   Toy Story 4|               en|       7.6|1073394593|
+--------------+-----------------+----------+----------+



In [70]:
# verificando a quantidade de languages
from pyspark.sql.functions import count, col, asc, desc
data_lang = dataset.groupBy('original_language').count()
data_lang.show()

+--------------------+-----+
|   original_language|count|
+--------------------+-----+
|                  en| 7798|
|                 101|    1|
|                  nb|    1|
| they are tasked ...|    1|
|               Roger|    1|
|                  ro|    2|
|                  ka|    1|
|                  ur|    1|
|                  lv|    3|
|                  pl|   19|
|                  cn|   79|
|                  sk|    1|
| Pierre could bec...|    1|
|                   0|    3|
|                  pt|   55|
|                null|   23|
|                  tl|    3|
|                  ko|  140|
|                  ms|    1|
|                  uk|    5|
+--------------------+-----+
only showing top 20 rows



In [77]:
data_lang.orderBy(col('count').desc()).show()

+-----------------+-----+
|original_language|count|
+-----------------+-----+
|               en| 7798|
|               ja|  609|
|               es|  417|
|               fr|  293|
|               ko|  140|
|               it|  104|
|               zh|  103|
|               cn|   79|
|               de|   78|
|               ru|   69|
|               pt|   55|
|               da|   30|
|               hi|   30|
|               no|   24|
|             null|   23|
|               sv|   22|
|               pl|   19|
|               nl|   19|
|               id|   15|
|               th|   15|
+-----------------+-----+
only showing top 20 rows



In [78]:
# filtrando algumas languages
dataset.filter(dataset.original_language.isin('pt', 'en', 'ja')).show()

+------------------------------------+-----------------+----------+----+
|                      original_title|original_language|mean_votes|time|
+------------------------------------+-----------------+----------+----+
|                Venom: Let There ...|               en|       6.8|  97|
|                            Eternals|               en|       7.1| 157|
|                                Dune|               en|       8.0| 155|
|                     Army of Thieves|               en|       6.9| 127|
|                            Free Guy|               en|       7.8| 115|
|                 Gunpowder Milkshake|               en|       6.5| 114|
|                               Venom|               en|       6.8| 112|
|                     Halloween Kills|               en|       7.0| 105|
|                     American Badger|               en|       6.3|  88|
|                 The Addams Family 2|               en|       7.4|  93|
|劇場版 七つの大罪 光に呪われし者たち|               ja|       8.4|n